# 內文相似度檢測

In [1]:
from gensim.models import word2vec
from gensim.models import Doc2Vec, doc2vec
from sklearn.cluster import KMeans
import pickle
import numpy as np
import pandas as pd
import os

## turn back to main directory
os.chdir("../")
os.getcwd()

'/project/at082-group17'

In [2]:
model = doc2vec.Doc2Vec.load('AT082_17_Leopard_Read/model/doc2vec')

In [3]:
mail = pd.read_csv('data/mail_preprocessed.csv')

In [4]:
#mail.head()
print(mail['content'][5])

禾宸生醫國際有限公司  台灣原生 山芙蓉 修護保濕霜 立即體驗用品申請 


In [5]:
## load 'article_cutted'
with open('data/mail_cutted', 'rb') as file:
    cut_data = pickle.load(file)

In [6]:
## create a document id map
sentence_list = []

for i, l in enumerate(cut_data):
    sentence_list.append(doc2vec.LabeledSentence(words=l, tags=[str(i)]))

In [11]:
sentence_list[2][0]

['禾宸',
 '生醫',
 '國際',
 '有限公司',
 '台灣',
 '原生',
 '山',
 '芙蓉',
 '修護',
 '保濕霜',
 '立即',
 '體驗',
 '用品',
 '申請']

In [12]:
def ceshi(sentence_list, count):
    #每次都要重新load model
    model_dm = doc2vec.Doc2Vec.load('AT082_17_Leopard_Read/model/doc2vec')
    inferred_vector_dm = model_dm.infer_vector(sentence_list)
    sims = model_dm.docvecs.most_similar([inferred_vector_dm], topn=count)
    print(sims)
    return sims

## 找出最相似的文件

In [13]:
sims = ceshi(sentence_list[5][0], 20)
for i in sims:
    print(i[0])
    print(mail['content'][int(i[0])])

[('5', 0.774357259273529), ('29130', 0.6906501650810242), ('10890', 0.6763108968734741), ('2936', 0.6594372391700745), ('6020', 0.6530652046203613), ('29797', 0.6467612385749817), ('7741', 0.6459895968437195), ('28080', 0.6070541143417358), ('13387', 0.5958379507064819), ('22084', 0.590429425239563), ('30479', 0.5884854793548584), ('9187', 0.5802339315414429), ('23446', 0.574857771396637), ('12364', 0.5727348327636719), ('9457', 0.5480268001556396), ('24519', 0.5444530248641968), ('444', 0.5444110631942749), ('9182', 0.5440943837165833), ('8463', 0.5435901284217834), ('30414', 0.541185736656189)]
5
禾宸生醫國際有限公司  台灣原生 山芙蓉 修護保濕霜 立即體驗用品申請 
29130
台灣精華食品 情滿中秋~送禮、送餅，不如送健康。台灣精華食品為您提供中秋佳節最健康的伴手禮
10890
獨角獸男仕保養  Mr. U 傳送了一則訊息 提醒您，購物車中的優惠將於３日內過期，敬請把握
2936
衛采購物  衛采購物周年慶 Chocola BB、優補利-富R通通69折起，只到12/4，膠原蛋白要補充更要守住，大家一起美麗迎歲末
6020
法蘭西娜-無負擔保養品  name  您好，得獎保養品 任選79折
29797
Candy Lee 台亞光電液晶面板,長條顯示器.觸控面板.高亮面板.驅動板.樹莓派- Candy Lee
7741
Milly 《拓展國際品牌行銷能見度，您該如何做？》B2B外銷實務座談會~最後3席!
28080
中華民國地球科學學會  活動訊息 9/3-4 第一屆氣

## 沒看過的測資測試

In [14]:
import jieba
import jieba.posseg as pseg
import re

In [15]:
## set dictionary (can define yourself)
jieba.set_dictionary('AT082_17_Leopard_Read/jieba/dict.txt.big')
stop_words = open('AT082_17_Leopard_Read/jieba/stop_words.txt').read().splitlines()

In [16]:
def cutTest(text):
    line = []
    text = text.replace('https?:\/\/\S*|︱', '')
    text = text.replace('【', ' ')
    text = text.replace('】', ' ')
    text = text.replace('{', ' ')
    text = text.replace('}', ' ')
    text = text.replace('$', '')
    text = text.replace('『', ' ')
    text = text.replace('』', ' ')
    for w in jieba.cut(text, cut_all=False):
        if w not in stop_words and not bool(re.match('[0-9]+', w)):
            line.append(w)
    return line

In [17]:
def showTop(text, count):
    sims = ceshi(cutTest(text), count)
    print(cutTest(text))
    indexList = []
    contentList = []
    similarityList = []
    for i in sims:
        indexList.append(i[0])
        similarityList.append(i[1])
        contentList.append(mail['content'][int(i[0])])
    df = pd.DataFrame()
    df['index'] = indexList
    df['similarity'] = similarityList
    df['content'] = contentList
    return df

In [19]:
import ipywidgets as widgets
from IPython.display import display

intputContent = widgets.Textarea(
    value='',
    placeholder='',
    description='mail content:',
    disabled=False
)

count =  widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Top N:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
output = widgets.Output()
def clickContent(b):
    with output:
        print('target :' + intputContent.value)
        df = showTop(intputContent.value, count.value)
        display(df)
        #df.head(count.value)
def changeInput(change):
    with output:
        output.clear_output()
        print('target :' + change['new'])
        intputContent.value = change['new']
def changeCount(change):
    with output:
        output.clear_output()
        count.value = change['new']
button = widgets.Button(description="analysis")
button.on_click(clickContent)
display(intputContent,count, button, output)
intputContent.observe(changeInput, names='value')
intputContent.observe(count, names='value')

Textarea(value='', description='mail content:', placeholder='')

IntSlider(value=10, continuous_update=False, description='Top N:', min=1)

Button(description='analysis', style=ButtonStyle())

Output()